In [ ]:
import numpy as np
import control as con
from numpy import linalg as LA

import cvxpy
import optim_tools as tools#own file with helper

In [ ]:
##############################
# Boris Paper UBoot          #
##############################
A = np.matrix([[0, 1, 0],
              [0, 0, 1],
              [0, 0, -0.005]])
a = -A[-1,:].T ### !!!!!
b = np.matrix([[0],[0],[1]])
c = np.matrix([1, 0, 0])
d = np.matrix([0])
u_max = 2.5e-5
n = 3

X0 = [np.matrix([-10, -0.05, -0.0046]).T,
      np.matrix([-10, -0.05, 0.0046]).T,
      np.matrix([-10, 0.05, -0.0046]).T,
      np.matrix([10, -0.05, 0.0046]).T,
      np.matrix([10, -0.05, -0.0046]).T,
      np.matrix([10, 0.05, 0.0046]).T]

#print "A:\n", A
#print "a:\n", a
#print "b:\n", b
#print "c:\n", c

##### Entwurf parameter #####
beta = 2 # beta >=1 !

In [ ]:
def get_S_list(u_max, Q, z, a, n):
    S_list = [cvxpy.bmat([[u_max**2 , z.T],
                         [z        , Q]])] # S0 is different !
    
    for i in range(n+1)[1:]:
        #print i
        #print -a[n-i]
        q = Q[:, n-i] # Slicing, (n-i)th column of Q!
        #print q
        S_list.append(-a[n-i] * cvxpy.bmat([[0, q.T],
                                            [q, np.zeros((n,n))]]))
    return S_list

S_list = get_S_list(u_max, Q, z, a, n)
S_list

In [ ]:
from scipy.special import comb as nchoosek # n Choose k (n ueber k)

# As seen in A.4 jasniwiecz
# Intervalltransformation (Gleichung (4.36))
# p => [p_l, p_u] (p => [p_min, 1]) wird zu p^ => [-1,1] return neue Matrizen(!) mit diesen p^
#
# S_in ist Liste aus Faktormatrizen für Polynom in p
# S_out ist Liste aus Faktormatrizen für Polynom in p^
# a = (p_u - p_l)
# b = (p_u + p_l)
def trans_S_list(S_in, pl=0.01, pu=1):
    a = pu - pl
    b = pu + pl
    n = len(S_in) # Anzahl der Matrizen in S_in

    S_out = [np.zeros(S_in[0].size)] * n
    for j in range(0, n): # Bearbeite jede Matrix
        for i in range(j, n): # Jeweils mit Summe der anderen
            S_out[j] = S_out[j] + 2**(-i) * b**(i-j) * nchoosek(i, j) * S_in[i]
        S_out[j] = a**j*S_out[j]
    return S_out

S_list_T = trans_S_list(S_list)

In [ ]:
def calc_S_Sum(S_list):
    l = len(S_list) # number of matrizen in S_list
    m = l-1 # Index of Q_m
    n = S_list[0].size[0] # shape of each matrix, first element
    
    if m is 0:
        S_sum = cvxpy.bmat([[2*S_list[0],   np.zeros(n)], 
                            [np.zeros(n), np.zeros(n)]])
    elif m is 1:
        S_sum = cvxpy.bmat([[2*S_list[0], S_list[1]],
                            [S_list[1],   np.zeros(n)]])
    else: # e.g. m is 2 or more
        S_sum = cvxpy.bmat([[2*S_list[0], S_list[1]],
                            [S_list[1], 2*S_list[2]]])

    for i1 in range(3, l, 2):
        S_new_col = cvxpy.vstack(np.zeros((((i1+1)/2-1)*n, n)), S_list[i1])

        if i1 is m:
            S_new_row = cvxpy.hstack(np.zeros((n, ((i1+1)/2-1)*n)), S_list[i1], np.zeros((n,n)))
        else:
            S_new_row = cvxpy.hstack(np.zeros((n, ((i1+1)/2-1)*n)), S_list[i1], 2*S_list[i1+1])

        S_sum = cvxpy.bmat([[S_sum, S_new_col],
                            [S_new_row]])

    S_sum = 0.5*S_sum
    return S_sum
calc_S_Sum(S_list_T)

In [ ]:
##############################
# Convex Problem (35)        #
##############################

# Variables
Q  = cvxpy.Semidef(n) # Implys (27a) (semidefinite for numerical reasons?)

z = cvxpy.Variable(n)
z1 = cvxpy.Variable(n) # z_{star}

P = cvxpy.Semidef(n) #positiv definite (semidefinite for numerical reasons?)
G = cvxpy.Variable(n,n) #skew

# Constants
N = tools._N(n)
A0 = A + b*a.T

# Parameters


# Constraints
constraint_27b = Q*N+N*Q << 0
constraint_27d = Q*A0.T + A0*Q - z*b.T - b*z.T << 0

constraint_skew = G + G.T == 0 # skew symmetry


In [ ]:
# S. 78 Boris (LMI-Entwurf)
def convex_problem(gamma, mu=1):
    ##############################
    # Convex Problem (35)        #
    ##############################
    prob = pic.Problem()

    # Constants
    AA = pic.new_param('A', A)
    II = pic.new_param('I_n', np.eye(n))
    III = pic.new_param('I_n+1', np.eye(n+1))
    aa = pic.new_param('a', a)
    bb = pic.new_param('b', b)
    XX0 = pic.new_param('X0', X0)

    NN = pic.new_param('N', N)
    MM = pic.new_param('M', M)

    AA0 = pic.new_param('A0', AA+bb*aa.T)

    ## REMARK THIS!!!! gamma is optimization variable but not convex, thus to be bisected to find "bigg-ish" value
    gamma = pic.new_param('gamma', gamma)

    # Problem
    prob = pic.Problem()

    # Parameters
    QQ = prob.add_variable('Q', (n, n), vtype='symmetric')
    zz = prob.add_variable('z', n)
    zz_star = prob.add_variable('z_star', n)

    # Objective
    prob.set_objective('find', None)

    # Constraints
    #prob.add_constraint(QQ >> 0)
    #prob.add_constraint(QQ*NN + NN*QQ << 0)
    prob.add_constraint(QQ*AA0.T + AA0*QQ - zz*bb.T - bb*zz.T << 0)

    ## (31) ???

    prob.add_list_of_constraints([((1          & XX0[i].T) //
                                   (XX0[i]     & QQ      )) >> 0
                                       for i in range(0, len(X0))])

    prob.add_constraint(QQ*AA0.T + AA0*QQ - zz_star*bb.T - bb*zz_star.T << -2*(gamma*QQ))
    prob.solve(verbose=0, solver='cvxopt')
    return prob

In [ ]:
## Lets bisect
# Expects min_val to be valid, and max_val to be not valid
# func only taking on (scalar) argument -> gamma
def bisect_prob(min_val, max_val, func, diff=1e-5, max_iter=50, _iteration=0):
    if _iteration > max_iter: 
        print "Iter:", _iteration
        print "Diff:", (max_val - min_val)/2.0
        return min_val, prob
    elif (max_val - min_val)/2.0 <= diff:
        print "Iter:", _iteration
        print "Diff:", (max_val - min_val)/2.0
        return min_val, prob
    else:
        mid_val = min_val+(max_val - min_val)/2.0
        #print "1. Evaluating: ", mid_val
        try:
            prob = func(mid_val)
            Q = prob.get_valued_variable('Q')
            z = prob.get_valued_variable('z')
            z_star = prob.get_valued_variable('z_star')
        except Exception as e:
            #print "Problem not solved!"
            #print e
            max_val = mid_val
        else:
            #print "Problem solved!"
            #print Q, z, z_star
            min_val = mid_val
        finally:
            #print "2. Recursing: ", min_val, max_val
            return bisect_prob(min_val, max_val, func, diff, max_iter,_iteration+1)
        #return min_val, max_val
    

In [ ]:
#%%timeit ~4sec
val, prob = bisect_prob(1, 50, convex_problem)
print val
print convex_problem(val).get_valued_variable('Q')

In [ ]:
prob = convex_problem(5.1)
Q = None
z = None
z_star = None
try:
    Q = prob.get_valued_variable('Q')
    z = prob.get_valued_variable('z')
    z_star = prob.get_valued_variable('z_star')
except Exception as e:
    print "Problem not solved!"
    print e
else:
    print "Problem solved!"
    print Q, z, z_star
#print prob.get_valued_variable('z')
#print prob.get_valued_variable('z_star')

#print "P:\n", PP
#print "eig:\n", LA.eigvals(PP.value)

#print 
#print "AP+PA:\n",(AA.T*PP + PP*AA).value
#print "eig:\n", LA.eigvals((AA.T*PP + PP*AA).value)

In [ ]:
## Seite 47 Boris
# Wähle p_min, mu_start, zeta, zeta_start
def step1(p_min, mu_start, zeta, zeta_start):
    # Löse (A.15) mit mu = 1 -> l_star -> lambda_hat_i(1)=lambda_i(A-b*l_star.T)
    